In [1]:
import requests
import pandas as pd
import numpy as np
import folium
from autobahn.autobahn import get_warnings
from autobahn.car_model import Car
from autobahn.autobahn import map_plot
from autobahn.autobahn import TrafficWarning
from autobahn.autobahn import calculate_traffic_length


In [2]:
url_bab = "https://verkehr.autobahn.de/o/autobahn"

warnings_a7 = get_warnings('A7')

In [3]:
osrm_url = "http://router.project-osrm.org/route/v1/driving/"
start = "9.346188,54.772891"  # Flensburg lng, lat
# end = "9.479746,51.312711"  # Kassel
end = "10.659156,47.568033"# Fuessen
route_url = f"{osrm_url}{start};{end}?overview=full&geometries=geojson"
route_response = requests.get(route_url)
route_data = route_response.json()
route_coords = route_data['routes'][0]['geometry']['coordinates']
route_df = pd.DataFrame(route_coords, columns=['long', 'lat'])

### OSM Data

In [4]:
print(f"distance: {route_data['routes'][0]['distance'] / 1000:.2f} km")
print(f"duration: {route_data['routes'][0]['duration'] / 3600:.2f} h")

distance: 957.34 km
duration: 9.45 h


In [5]:
car = Car(start_location=(54.793743, 9.446996))  # Flensburg
for coord in route_coords:
    car.drive_to((coord[1], coord[0]))

car.distance_traveled

964.2204448053376

In [6]:
route_df = route_df.assign(speed=120)
map_plot([route_df], on="speed")

c:\python\kassel\3_social_coding\.env\lib\site-packages\folium\utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])


### Assign traffic information

In [7]:
[print(data["title"]) for data in warnings_a7]

A7 | Ellwangen - Aalen/Westhausen
A7 | Rehhecke - Riedenberg
A7 | Uttrichshausen - Lohmühle
A7 | Lohmühle - Am Forsthaus
A7 | Steinborntal - Lohmühle
A7 | Petersberg - Rauschenberg
A7 | Hildesheim - Derneburg/Salzgitter
A7 | Schneverdingen - Soltau-Ost
A7 | Hamburg-Volkspark - Hamburg-Schnelsen-Nord
A7 | Brokenlande - Neumünster-Mitte


[None, None, None, None, None, None, None, None, None, None]

In [10]:
from haversine import haversine, haversine_vector

threshold = .1 # km

In [11]:
for warning in warnings_a7:
    traffic_warning = TrafficWarning(warning)

    gps_coords_warnings = [(row1['lat'], row1['long']) for _, row1 in traffic_warning.geo_df.iterrows()]
    gps_coords_route = [(row2['lat'], row2['long']) for _, row2 in route_df.iterrows()]
    distance_matrix = haversine_vector(gps_coords_warnings, gps_coords_route, comb=True)
    if distance_matrix.min() < threshold:
        print(f"{traffic_warning.title} with delay: {traffic_warning.delayTimeValue}, speed: {traffic_warning.averageSpeed}, distance: {calculate_traffic_length(traffic_warning.geo_df)}")
        ind_route_traffic = route_df["speed"].where(distance_matrix.min(axis=1) < threshold).dropna().index
        route_df.loc[ind_route_traffic, "speed"] = float(traffic_warning.averageSpeed) if traffic_warning.averageSpeed else 80
    # break
    # print(f"Warning: {traffic_warning.title} is part of route: {is_part_of_route}")



A7 | Ellwangen - Aalen/Westhausen with delay: 2, speed: 35.599998474121094, distance: 2.310296201225475


C:\Users\krd2rng\AppData\Local\Temp\ipykernel_20808\2974961059.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35.599998474121094' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  route_df.loc[ind_route_traffic, "speed"] = float(traffic_warning.averageSpeed) if traffic_warning.averageSpeed else 80


A7 | Rehhecke - Riedenberg with delay: 0, speed: 49.599998474121094, distance: 0.9506326993517084
A7 | Uttrichshausen - Lohmühle with delay: None, speed: None, distance: 0.19006952748413344
A7 | Lohmühle - Am Forsthaus with delay: None, speed: None, distance: 3.461186812159217
A7 | Steinborntal - Lohmühle with delay: None, speed: None, distance: 0.3888327327668872
A7 | Petersberg - Rauschenberg with delay: 0, speed: 66.19999694824219, distance: 1.1644281706479993
A7 | Hildesheim - Derneburg/Salzgitter with delay: 5, speed: 24.5, distance: 2.5492750899854384
A7 | Schneverdingen - Soltau-Ost with delay: None, speed: None, distance: 8.31732621993373
A7 | Hamburg-Volkspark - Hamburg-Schnelsen-Nord with delay: 19, speed: 18.200000762939453, distance: 7.242916522546128
A7 | Brokenlande - Neumünster-Mitte with delay: 4, speed: 25.5, distance: 2.4570092117199924


In [15]:
import plotly.express as px
px.line(route_df["speed"])